<a href="https://colab.research.google.com/github/SEOUL-ABSS/Sonya/blob/main/TEST2_YEEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# Colab에서 실행하는 배 소음 탐지 통합 노트북
# - 사내망 환경 (TF 2.12, Python 3.10)에 맞춰 버전 고정
# - 오프라인 사용을 위한 모든 자원 다운로드 및 패키징 포함
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. 환경 설정: 필요한 라이브러리 설치
# ------------------------------------------------------------------------------
print("1. 환경 설정 시작: 필요한 라이브러리 설치 중...")

# TensorFlow 2.12.0을 포함한 모든 라이브러리를 정확한 버전으로 설치
!pip install tensorflow==2.12.0
!pip install tensorflow-hub==0.16.1
!pip install librosa==0.10.1
!pip install soundfile==0.12.1
!pip install numpy==1.24.4
!pip install pandas==2.2.2
!pip install scikit-learn==1.5.0
!pip install matplotlib==3.9.0
!pip install seaborn==0.13.2

print("\n라이브러리 설치 완료. 런타임 재시작이 필요할 수 있습니다.")
print("설치된 TensorFlow 버전 확인:", tf.__version__)


# ------------------------------------------------------------------------------
# 2. 자원 다운로드
# ------------------------------------------------------------------------------
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import soundfile as sf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

print("\n2. 자원 다운로드 시작...")

# DeepShip 데이터셋 클론
if not os.path.exists('/content/DeepShip'):
    !git clone https://github.com/irfankamboh/DeepShip.git
    print("DeepShip 데이터셋 클론 완료.")
else:
    print("DeepShip 데이터셋이 이미 존재합니다.")

# YAMNet 모델 다운로드
# Colab은 자동으로 /tmp/tfhub_modules에 캐시하므로 경로 설정이 필요 없음
try:
    print("YAMNet 모델 다운로드 중 (Colab 캐시 사용)...")
    yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
    yamnet_model = hub.load(yamnet_model_handle)
    YAMNET_SAMPLE_RATE = 16000
    YAMNET_EMBEDDING_DIM = 1024
    print("YAMNet 모델 다운로드 완료.")
except Exception as e:
    print(f"오류: YAMNet 모델 다운로드 중 오류 발생: {e}")
    sys.exit(1)


# ------------------------------------------------------------------------------
# 3. 학습 예제 실행 (이전 코드와 동일한 로직)
# ------------------------------------------------------------------------------
print("\n3. 학습 예제 실행 시작...")

DEEPSHIP_ACOUSTIC_DATA_PATH = '/content/DeepShip/Acoustic_data'

def preprocess_audio_for_yamnet(audio_path, yamnet_model, target_sample_rate=YAMNET_SAMPLE_RATE):
    try:
        waveform, sr = sf.read(audio_path, dtype='float32')
        if sr != target_sample_rate:
            waveform = librosa.resample(y=waveform, orig_sr=sr, target_sr=target_sample_rate)
        if waveform.ndim > 1:
            waveform = np.mean(waveform, axis=1)
        scores, embeddings, spectrogram = yamnet_model(tf.constant(waveform, dtype=tf.float32))
        mean_embedding = tf.reduce_mean(embeddings, axis=0)
        return mean_embedding.numpy()
    except Exception as e:
        print(f"오디오 파일 처리 중 오류 발생: {audio_path}, 오류: {e}")
        return None

all_audio_paths = []
all_labels = []
sub_data_folders = ['Train', 'Test']
for sub_folder in sub_data_folders:
    current_data_path = os.path.join(DEEPSHIP_ACOUSTIC_DATA_PATH, sub_folder)
    if os.path.exists(current_data_path) and os.path.isdir(current_data_path):
        for class_name in os.listdir(current_data_path):
            class_path = os.path.join(current_data_path, class_name)
            if os.path.isdir(class_path):
                for file_name in os.listdir(class_path):
                    if file_name.endswith('.wav'):
                        audio_path = os.path.join(class_path, file_name)
                        all_audio_paths.append(audio_path)
                        all_labels.append(class_name)

if not all_audio_paths:
    print("오류: 데이터 파일을 찾을 수 없어 학습을 진행할 수 없습니다. 스크립트를 종료합니다.")
    sys.exit(1)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(all_labels)
num_classes = len(label_encoder.classes_)

X_train_paths, X_test_paths, y_train_encoded, y_test_encoded = train_test_split(
    all_audio_paths, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

X_train_embeddings = np.array([preprocess_audio_for_yamnet(p, yamnet_model) for p in X_train_paths if preprocess_audio_for_yamnet(p, yamnet_model) is not None])
X_test_embeddings = np.array([preprocess_audio_for_yamnet(p, yamnet_model) for p in X_test_paths if preprocess_audio_for_yamnet(p, yamnet_model) is not None])
y_train_one_hot = tf.keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)[:len(X_train_embeddings)]
y_test_one_hot = tf.keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)[:len(X_test_embeddings)]

model = Model(
    inputs=Input(shape=(YAMNET_EMBEDDING_DIM,)),
    outputs=Dense(num_classes, activation='softmax')(Dropout(0.5)(Dense(128, activation='relu')(Dropout(0.5)(Dense(256, activation='relu')(Input(shape=(YAMNET_EMBEDDING_DIM,)))))))
)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

history = model.fit(
    X_train_embeddings, y_train_one_hot,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

loss, accuracy = model.evaluate(X_test_embeddings, y_test_one_hot, verbose=0)
print(f"\n모델 평가 완료. 테스트 세트 정확도: {accuracy:.4f}")


# ------------------------------------------------------------------------------
# 4. 오프라인 사용을 위한 자원 패키징
# ------------------------------------------------------------------------------
print("\n4. 오프라인 사용을 위한 자원 패키징 시작...")

# 최종 ZIP 파일 이름
zip_file_name = 'offline_resources.zip'

# 패키징을 위한 디렉토리 생성
os.makedirs('./offline_resources/python_packages', exist_ok=True)

# requirements.txt 파일 생성
requirements_content = """
tensorflow==2.12.0
tensorflow-hub==0.16.1
librosa==0.10.1
soundfile==0.12.1
numpy==1.24.4
pandas==2.2.2
scikit-learn==1.5.0
matplotlib==3.9.0
seaborn==0.13.2
"""
with open('./offline_resources/requirements.txt', 'w') as f:
    f.write(requirements_content.strip())

# pip download를 이용해 패키지 다운로드
!pip download -r ./offline_resources/requirements.txt -d ./offline_resources/python_packages

# YAMNet 모델 캐시 폴더 복사
yamnet_cache_dir = os.environ.get('TFHUB_CACHE_DIR', '/tmp/tfhub_modules')
!cp -r {yamnet_cache_dir} ./offline_resources/yamnet_model

# DeepShip 데이터셋 복사
!cp -r /content/DeepShip ./offline_resources/DeepShip_dataset

# 모든 자원을 ZIP 파일로 압축
print(f"\n'{zip_file_name}' 파일로 압축 중...")
!zip -r {zip_file_name} ./offline_resources

print("\n=====================================================")
print("         모든 자원 패키징 완료!")
print("=====================================================")
print(f"'{zip_file_name}' 파일을 로컬로 다운로드하세요.")
print("이 파일은 Colab 왼쪽의 파일 탐색기에서 찾을 수 있습니다.")

# Colab에서 다운로드 프롬프트를 띄우는 코드
from google.colab import files
files.download(zip_file_name)